# Endless Content Recommendation Engine
This is a prototype content recommendation engine for Endless OS Apps which is based off of the fast.ai-v2 lesson on collaborative filtering. Instead of ratings, it uses and predicts a "content recency factor" which is a linear decay of how recently a user accessed a piece of content (1.0 == user accessed content just now, 0.0 == user never accessed content).

Disclaimer: Even though I work for Endless, this is not an official Endless product. This notebook and associated files are essentially the work of an software engineer tinkering over the weekend.

In [1]:
from fastai.learner import *
from fastai.column_data import *

In [2]:
import os

## Content spreadsheet
The content spreadsheet (generated by `dump_apps.py`) contains a row for each item and some basic information about that item (what app it came from, the first three words of its title and any non-internal tags it has). We'll use this information later in order to learn what information is relevant to the user's preferences.

In [4]:
content = pd.read_csv(os.path.join('..', 'data', 'content.csv'))

In [5]:
# Show the first few rows
content.head()

,itemId,appId,titleWord1,titleWord2,titleWord3,tag1,tag2,tag3
0,com.endlessm.animals.en/0048dc046d98af91fba689...,com.endlessm.animals.en,Manx,Cat,NaN,Mammals,NaN,NaN
1,com.endlessm.animals.en/00dbddffbadc2915139296...,com.endlessm.animals.en,Sheep,NaN,NaN,Mammals,NaN,NaN
2,com.endlessm.animals.en/0110c2aedf52a77a3182f5...,com.endlessm.animals.en,Boston,Terrier,NaN,Mammals,NaN,NaN
3,com.endlessm.animals.en/01e8909ad4987ab8f2d112...,com.endlessm.animals.en,Bombyx,Mori,NaN,Insects,NaN,NaN
4,com.endlessm.animals.en/02b6756b18cef13ff99a75...,com.endlessm.animals.en,Tadpole,NaN,NaN,Amphibians,NaN,NaN


## Access spreadsheet
The access spreadsheet is a record of how recently each piece of content was accessed. If a piece of content does not appear in this spreadsheet you can assume that it was never accessed. The `itemId` column corresponds to an `itemId` in `content.csv` and `timestamp` is the unix timestamp of when the content was last accessed (higher timestamps are more recent).

In [6]:
accesses = pd.read_csv(os.path.join('..', 'data', 'access.csv'))

In [7]:
accesses.head()

,itemId,timestamp
0,com.endlessm.animals.en/d33c55379ace2a870e1e03...,1526212659
1,com.endlessm.celebrities.en/44cedd6ff0b4a3d07e...,1526212681
2,com.endlessm.celebrities.en/d541d9cc02ad74a7f4...,1526201373
3,com.endlessm.animals.en/3990a2d2241af8fda2836b...,1526201320
4,com.endlessm.celebrities.en/a312e07980c15fe7c5...,1526215827


In [8]:
# Outer-join the content spreadsheet with the access spreadsheet on itemId. All rows in both spreadsheets
# will be included, however timestamps for entries that were never accessed will be set to NaN.
content_accesses = content.set_index('itemId').join(accesses.set_index('itemId'), how='outer', lsuffix='_l', rsuffix='_r')

In [9]:
# Show the 50 most recent entries sorted by time of access descending.
content_accesses.sort_values('timestamp', ascending=False).head(50)

,appId,titleWord1,titleWord2,titleWord3,tag1,tag2,tag3,timestamp
itemId,,,,,,,,
com.endlessm.celebrities.en/884ab75f81d0ed57bfab68458663bf31810e2f3f,com.endlessm.celebrities.en,José,Mujica,NaN,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/382d9ee00576f0f79524c0cd00f7eee464dd1a64,com.endlessm.celebrities.en,Kim,Jong-Un,NaN,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/ecc7b2ffbd81b7b4850f2d0774856dfcb0a7c74f,com.endlessm.celebrities.en,Angela,Merkel,NaN,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/ad9b057b1f957f2ec3698409ba755ae94d0cee8a,com.endlessm.celebrities.en,John,F.,Kennedy,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/a312e07980c15fe7c52dc65fb25e4a00191c0ca8,com.endlessm.celebrities.en,Emiliano,Zapata,NaN,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/9d695ea5505110dbd3b3910fb960f5cb1c8a5436,com.endlessm.celebrities.en,Franklin,D.,Roosevelt,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/508dafa056c37a72fe9a292976118c354bb6c106,com.endlessm.celebrities.en,Pancho,Villa,NaN,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/1e03b8af1f43f33ba8fdaa9ce685b42dc1bf74e9,com.endlessm.celebrities.en,Cristina,Fernández,de,Politicians,NaN,NaN,1.526216e+09
com.endlessm.celebrities.en/f9b832875bfe166fc87f0527077b2e6435755daf,com.endlessm.celebrities.en,Jorge,Serrano,Elías,Politicians,NaN,NaN,1.526216e+09


## Conversion to Decay Ranges
Trying to predict in the 10^9 domain of timestamps is a little bit useless, so we'll convert from that domain into the unit scale, where 1 is the most recently accessed and 0 is the least recently accessed or never accessed.

This is a pretty straightforward computation - just subtract the minimum, then scale by the inverse of the range. Note that all the NaN entries get a "0" for their timestamp value now.

In [10]:
decay_ranges = content_accesses.timestamp.min(), content_accesses.timestamp.max()

In [11]:
decay_range = decay_ranges[1] - decay_ranges[0]

In [12]:
content_decay = content_accesses.copy()
content_decay.timestamp = content_accesses.timestamp.fillna(decay_ranges[0]).apply(lambda x: (x - decay_ranges[0]) / decay_range)

In [13]:
# Show entries by recency (unit scale)
content_decay.sort_values('timestamp', ascending=False).head(50)

,appId,titleWord1,titleWord2,titleWord3,tag1,tag2,tag3,timestamp
itemId,,,,,,,,
com.endlessm.celebrities.en/884ab75f81d0ed57bfab68458663bf31810e2f3f,com.endlessm.celebrities.en,José,Mujica,NaN,Politicians,NaN,NaN,1.000000
com.endlessm.celebrities.en/382d9ee00576f0f79524c0cd00f7eee464dd1a64,com.endlessm.celebrities.en,Kim,Jong-Un,NaN,Politicians,NaN,NaN,0.999725
com.endlessm.celebrities.en/ecc7b2ffbd81b7b4850f2d0774856dfcb0a7c74f,com.endlessm.celebrities.en,Angela,Merkel,NaN,Politicians,NaN,NaN,0.998211
com.endlessm.celebrities.en/a312e07980c15fe7c52dc65fb25e4a00191c0ca8,com.endlessm.celebrities.en,Emiliano,Zapata,NaN,Politicians,NaN,NaN,0.998142
com.endlessm.celebrities.en/ad9b057b1f957f2ec3698409ba755ae94d0cee8a,com.endlessm.celebrities.en,John,F.,Kennedy,Politicians,NaN,NaN,0.998142
com.endlessm.celebrities.en/9d695ea5505110dbd3b3910fb960f5cb1c8a5436,com.endlessm.celebrities.en,Franklin,D.,Roosevelt,Politicians,NaN,NaN,0.998005
com.endlessm.celebrities.en/508dafa056c37a72fe9a292976118c354bb6c106,com.endlessm.celebrities.en,Pancho,Villa,NaN,Politicians,NaN,NaN,0.997798
com.endlessm.celebrities.en/1e03b8af1f43f33ba8fdaa9ce685b42dc1bf74e9,com.endlessm.celebrities.en,Cristina,Fernández,de,Politicians,NaN,NaN,0.997661
com.endlessm.celebrities.en/f9b832875bfe166fc87f0527077b2e6435755daf,com.endlessm.celebrities.en,Jorge,Serrano,Elías,Politicians,NaN,NaN,0.995941


In [14]:
# Truncate timestamps if they would be too old. We'll treat anything that we haven't
# looked at in a while as something that has never been looked at before.
content_decay.timestamp = content_decay.timestamp.apply(lambda x: 0 if x < 0.1 else x)

## Converting unique IDs to sequential IDs
In a lot of structured data, we have unique category identifiers (which may be numbers or strings) which we'll need to look up in our embedding tables later. In order to do that efficiently, we need to convert every unique ID to an offset or a sequential ID. The function below will do that for us by creating a map from some identifier to a sequential ID. Note that it is the caller's responsibility to make sure the identifier list is deduplicated first, otherwise the returned IDs will be nonsequential.

In [15]:
def sequential_lookup_tables(identifiers):
    identifier_to_sequential_id = {
        i: s for s, i in enumerate(identifiers)
    }
    sequential_id_to_identifier = list(identifiers)
    return identifier_to_sequential_id, sequential_id_to_identifier

## Converting categorical columns to sequential IDs
In this dataset we have a lot of columns that are essentially categories (appId can be one of `com.endlessm.animals.en`, `com.endlessm.celebrities.en`, `com.endlessm.travel.en`, `com.endlessm.history.en`, `com.endlessm.geography.en`, `com.endlessm.video_movement`, for instance). In order to treat these as offsets, lets convert them into sequential IDs by creating tables that can translate back and forth between the two.

In [16]:
categorical_columns = ['appId', 'titleWord1', 'titleWord2', 'tag1']

In [17]:
categorical_lookups = {
    column: {
        'sequential': to_sequential,
        'id': to_id
    }
    for column, to_sequential, to_id in [
        (column, *sequential_lookup_tables(content_decay[column].fillna('').unique()))
        for column in categorical_columns
    ]
}

In [18]:
categorical_lookups

{'appId': {'sequential': {'com.endlessm.animals.en': 0,
   'com.endlessm.celebrities.en': 1,
   'com.endlessm.geography.en': 2,
   'com.endlessm.history.en': 3,
   'com.endlessm.travel.en': 4,
   'com.endlessm.video_movement': 5},
  'id': ['com.endlessm.animals.en',
   'com.endlessm.celebrities.en',
   'com.endlessm.geography.en',
   'com.endlessm.history.en',
   'com.endlessm.travel.en',
   'com.endlessm.video_movement']},
 'titleWord1': {'sequential': {'Manx': 0,
   'Sheep': 1,
   'Boston': 2,
   'Bombyx': 3,
   'Tadpole': 4,
   'Red': 5,
   'Hemiptera': 6,
   'Fila': 7,
   'Yacare': 8,
   'White': 9,
   'Atlantic': 10,
   'Persian': 11,
   'Bed': 12,
   'Sighthound': 13,
   'Siamese': 14,
   'Asilidae': 15,
   'Spectacled': 16,
   'Killer': 17,
   'Cougar': 18,
   'Sugar': 19,
   'Common': 20,
   'Snake': 21,
   'Conch': 22,
   'Leatherback': 23,
   'Southern': 24,
   'Pit': 25,
   'Neapolitan': 26,
   'True': 27,
   'Orinoco': 28,
   'British': 29,
   'Glass': 30,
   'Caiman': 31,


In [19]:
# Get some validation indices to hold back from content_decay.
# We'll use this when constructing our dataset to work out what
# to validate against.
validation_indices = get_cv_idxs(len(content_decay))

In [20]:
# Turn named categorical variables into sequential numbers which are
# offsets into embeddings that we'll create later.
content_decay_indexed = content_decay.copy()
for column in categorical_columns:
    content_decay_indexed[column] = content_decay_indexed[column].fillna('').apply(lambda x: categorical_lookups[column]['sequential'][x])

## Create Dataset
Now that we've turned out dataset into something we can do machine learning on, create a fast.ai `Dataset` object. We'll do this using the `ColumnarModelData.from_data_frame` method as it allows us to separate out the categorical columns from the contiguous columns.

We have a couple of independent variables, namely the offsets for `appId`, `titleWord1`, `titleWord2` and `tag1` (we're ignoring the other title words and tags since they rarely came up in the dataset). The dependent variable is the "timestamp" which in this case will be the recency factor between 0 and 1.

In [21]:
independent_variables = content_decay_indexed.drop(['timestamp', 'titleWord3', 'tag2', 'tag3'], axis=1)
dependent_data = content_decay_indexed['timestamp'].astype(np.float32)
data = ColumnarModelData.from_data_frame(
    os.path.join('..', 'data', 'content.csv'),
    validation_indices,
    independent_variables,
    dependent_data,
    categorical_columns,
    bs=4
)

## Embeddings
Initialize the embeddings using uniformly distributed weights.

In [22]:
def create_uniformly_initialized_embedding(n_items, n_factors, range_min, range_max):
    embedding = nn.Embedding(n_items, n_factors)
    embedding.weight.data.uniform_(range_min, range_max)
    return embedding

In [23]:
def yield_slices(indexable, slice_length):
    '''Yield slices from an indexable in a sliding window'''
    for i in range(0, len(indexable) - (slice_length - 1)):
        yield indexable[i:i + slice_length]

# Neural Net
A Neural Network is basically a series of matrix multiplications interleaved with non-linear function applications (such that the elements in each matrix of each layer has a particular meaning).

Imagine that each content record is vector of properties such that when it is fed through the network a "recency factor" comes out the other end.

In our case the vector of properties looks something like this:

(com.endlessm.animals.en (1), Amphibious (2), Velociraptor (4), Reptiles (1)) -> (0.00026)

Or in other words (1, 2, 4, 1) -> (0.00026)

Now, the network "learns" parameters at a much higher degree of dimensionality than that, so how exactly does it do it? Well, each of the "categorical" variables gets passed through something called an "embedding" first. The "embedding" is a vector of properties that we learn for that category. So, if the embedding for apps has 10 timensions, and the embedding for words has 7 dimensions and the embedding for tags has 10 dimensions, what you actually end up with is something like this:

(1, 2, 4, 1) -> (
    (0.3423 ... (10) ... 0.1124),
    (0.3522 ... (7) ... 0.3991),
    (0.7878 ... (7) ... 0.5687),
    (0.1241 ... (10) ... 0.1299)
) -> (0.00026)

Note that the embeddings are initialized with a uniform distribution of values to start with, so passing the result of the embedding lookup through the network is usually going to result in a pretty rubbish answer. However, through error backpropagation those embeddings will eventually "correct" such that they "work out" better in all the training cases.

Now for the middle layer. This is the "hidden layer" where we can learn some more parameters to fit the function between the two non-linearities (in this case, the non-linearities are "Rectified Linear Units" or "ReLUs", which is just a fancy way of saying that we truncate all negative numbers to 0).

The hidden layer has 120 parameters. This means that there are 120 different weights associated with each of the 34 parameters that came from our embeddings. We'll learn each of these 120 weights for each of the 34 parameters such that no matter what 34 parameters come in from the embedding layer, we get a result that minimizes our later loss function.

Finally, the each of the 120 activations in the hidden layer gets multiplied by one weight and then summed together into the final layer, before it gets passed through a "sigmoid" function. The output of the sigmoid function is between -1 and 1 such that higher values get squished towards 1 and lower values get squished towards -1. In our case, we actually multiply the sigmoid output by 2 and then apply a ReLU to it, since our domain was between 0 and 1 anyway and we want the output of the network to be as close as possible to the domain of the regression taget so as to minimize the loss.

In [24]:
class EmbeddingNeuralNet(nn.Module):
    def __init__(self,
                 n_apps,
                 n_title_word1,
                 n_title_word2,
                 n_tag1,
                 hidden_layer_parameters=[120, 1],
                 dropout_probabilities=[0.05, 0.05]):
        super().__init__()
        self.app_embedding = create_uniformly_initialized_embedding(n_apps, 10, 0, 0.05)
        self.title_word1_embedding = create_uniformly_initialized_embedding(n_title_word1, 7, 0, 0.05)
        self.title_word2_embedding = create_uniformly_initialized_embedding(n_title_word2, 7, 0, 0.05)
        self.tag1_embedding = create_uniformly_initialized_embedding(n_tag1, 10, 0, 0.05)
        
        # The numbers here correspond to the dimensionality of
        # each of the embeddings. linear_layer_dimensions here essentially specifies
        # the number of activations on each layer of the network. First layer
        # after embeddings has 24 activations. Then we have 120. Then 1.
        linear_layer_dimensions = [10 + 7 * 2 + 10 * 1] + hidden_layer_parameters
        self.linear_layers = [
            nn.Linear(input_dimensions, output_dimensions)
            for input_dimensions, output_dimensions in
            yield_slices(linear_layer_dimensions, 2)
        ]
        # Apply dropout to each layer, which is a technique to prevent overfitting.
        #
        # Dropout essentially "kills" certain activations during training, forcing other
        # weights to adapt.
        self.dropout_layers = [
            nn.Dropout(probability)
            for probability in dropout_probabilities
        ]
        # We use ReLU for all the activations except for the last one, where we use
        # a sigmoid as explained above.
        self.activations = [
            F.relu for i in range(0, len(self.linear_layers) - 1)
        ] + [lambda x: F.relu(F.sigmoid(x) * 2)]
    
    def forward(self, categorical_variables, contiguous_variables):
        # Each of these are "mini-batches" or (batch_size (4)x1) vectors of categories. We pretend
        # that this is all of our training data where we accumulate gradients for the optimizer
        # to make a step. In reality, this is very little of our training data, but doing it this
        # way makes training go much faster.
        #
        # Doing it this way also allows us to do things like normalize amongst the
        # mini-batch.
        app_ids, title_word1s, title_word2s, tag1s, = [categorical_variables[:,i] for i in range(0, 4)]
        
        # torch.cat here basically takes all the embedding vectors and pastes them together
        # into one big long vector as explained above. We then feed that pasted vector
        # forward through the network.
        x = torch.cat([
            self.app_embedding(app_ids),
            self.title_word1_embedding(title_word1s),
            self.title_word2_embedding(title_word2s),
            self.tag1_embedding(tag1s)
        ], dim=1)
        
        # Apply dropout to the last activation, then apply the weights, then
        # apply the activation function.
        for layer, dropout, activation in zip(self.linear_layers, self.dropout_layers, self.activations):
            x = activation(layer(dropout(x)))

        return x;

In [25]:
# The size of each embedding depends on the number of its unique categories. In
# reality this doesn't impact too much since embeddings are implemented using
# an array offset anyway.
model = EmbeddingNeuralNet(int(independent_variables.appId.nunique()),
                           int(independent_variables.titleWord1.nunique()),
                           int(independent_variables.titleWord2.nunique()),
                           int(independent_variables.tag1.nunique()))
# "Adam" here is "Adaptive Momentum". It is basically stochastic gradient descent
# with a bit of momentum applied to get out of local minima.
optimizer = optim.Adam(model.parameters(), 1e-1, weight_decay=1e-5)

In [26]:
# Fit the model to the data. We run for 10 epochs.
fit(model, data, 10, optimizer, F.mse_loss)

  2%|▏         | 5/312 [00:00<00:35,  8.59it/s, loss=tensor(0.9373)]

/home/smspillaz/machinelearning/virtualenv/lib/python3.6/site-packages/fastai/model.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  return raw_loss.data[0]


 48%|████▊     | 150/312 [00:03<00:03, 48.06it/s, loss=tensor(1.00000e-02 *
 48%|████▊     | 150/312 [00:03<00:03, 47.79it/s, loss=tensor(1.00000e-02 *
 48%|████▊     | 150/312 [00:03<00:03, 47.53it/s, loss=tensor(1.00000e-02 *
 48%|████▊     | 150/312 [00:03<00:03, 47.28it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 49.68it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 49.42it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 49.21it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 48.95it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 48.70it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 48.45it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 48.17it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 47.92it/s, loss=tensor(1.00000e-02 *
 51%|█████     | 158/312 [00:03<00:03, 47.69it/s, loss=tensor(1.00000e-02 *
 53%|█████▎ 

 78%|███████▊  | 244/312 [00:04<00:01, 49.86it/s, loss=tensor(1.00000e-02 *
 78%|███████▊  | 244/312 [00:04<00:01, 49.68it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 250/312 [00:04<00:01, 50.81it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 250/312 [00:04<00:01, 50.65it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 250/312 [00:04<00:01, 50.41it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 250/312 [00:04<00:01, 50.21it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 250/312 [00:04<00:01, 50.03it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 250/312 [00:05<00:01, 49.87it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 250/312 [00:05<00:01, 49.71it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 256/312 [00:05<00:01, 50.78it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 256/312 [00:05<00:01, 50.61it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 256/312 [00:05<00:01, 50.45it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 256/312 [00:05<00:01, 50.28it/s, loss=tensor(1.00000e-02 *
 82%|███████

  9%|▊         | 27/312 [00:00<00:06, 47.37it/s, loss=tensor(1.00000e-02 *
  9%|▊         | 27/312 [00:00<00:06, 46.66it/s, loss=tensor(1.00000e-02 *
  9%|▊         | 27/312 [00:00<00:06, 45.54it/s, loss=tensor(1.00000e-02 *
  9%|▊         | 27/312 [00:00<00:06, 44.70it/s, loss=tensor(1.00000e-02 *
  9%|▊         | 27/312 [00:00<00:06, 43.91it/s, loss=tensor(1.00000e-02 *
  9%|▊         | 27/312 [00:00<00:06, 43.14it/s, loss=tensor(1.00000e-02 *
  9%|▊         | 27/312 [00:00<00:06, 42.39it/s, loss=tensor(1.00000e-02 *
  9%|▊         | 27/312 [00:00<00:06, 41.67it/s, loss=tensor(1.00000e-02 *
 12%|█▏        | 36/312 [00:00<00:04, 55.32it/s, loss=tensor(1.00000e-02 *
 12%|█▏        | 36/312 [00:00<00:05, 54.23it/s, loss=tensor(1.00000e-02 *
 12%|█▏        | 36/312 [00:00<00:05, 53.49it/s, loss=tensor(1.00000e-02 *
 12%|█▏        | 36/312 [00:00<00:05, 52.46it/s, loss=tensor(1.00000e-02 *
 12%|█▏        | 36/312 [00:00<00:05, 51.63it/s, loss=tensor(1.00000e-02 *
 12%|█▏        | 36/312 [

...

 79%|███████▊  | 245/312 [00:04<00:01, 50.15it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 251/312 [00:04<00:01, 51.29it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 251/312 [00:04<00:01, 51.10it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 251/312 [00:04<00:01, 50.92it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 251/312 [00:04<00:01, 50.74it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 251/312 [00:04<00:01, 50.57it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 251/312 [00:04<00:01, 50.37it/s, loss=tensor(1.00000e-02 *
 80%|████████  | 251/312 [00:04<00:01, 50.21it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 257/312 [00:05<00:01, 51.33it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 257/312 [00:05<00:01, 51.16it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 257/312 [00:05<00:01, 51.00it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 257/312 [00:05<00:01, 50.83it/s, loss=tensor(1.00000e-02 *
 82%|████████▏ | 257/312 [00:05<00:01, 50.66it/s, loss=tensor(1.00000e-02 *
 82%|███████

 10%|▉         | 30/312 [00:00<00:06, 40.99it/s, loss=tensor(1.00000e-02 *
 10%|▉         | 30/312 [00:00<00:07, 39.81it/s, loss=tensor(1.00000e-02 *
 10%|▉         | 30/312 [00:00<00:07, 38.82it/s, loss=tensor(1.00000e-02 *
 10%|▉         | 30/312 [00:00<00:07, 37.96it/s, loss=tensor(1.00000e-02 *
 10%|▉         | 30/312 [00:00<00:07, 37.12it/s, loss=tensor(1.00000e-02 *
 11%|█         | 35/312 [00:00<00:06, 42.87it/s, loss=tensor(1.00000e-02 *
 11%|█         | 35/312 [00:00<00:06, 42.00it/s, loss=tensor(1.00000e-02 *
 11%|█         | 35/312 [00:00<00:06, 41.21it/s, loss=tensor(1.00000e-02 *
 11%|█         | 35/312 [00:00<00:06, 40.44it/s, loss=tensor(1.00000e-02 *
 11%|█         | 35/312 [00:00<00:06, 39.72it/s, loss=tensor(1.00000e-02 *
 11%|█         | 35/312 [00:00<00:07, 38.98it/s, loss=tensor(1.00000e-02 *
 11%|█         | 35/312 [00:00<00:07, 38.30it/s, loss=tensor(1.00000e-02 *
 13%|█▎        | 41/312 [00:00<00:06, 44.48it/s, loss=tensor(1.00000e-02 *
 13%|█▎        | 41/312 [

 39%|███▉      | 123/312 [00:02<00:03, 48.66it/s, loss=tensor(1.00000e-02 *
 39%|███▉      | 123/312 [00:02<00:03, 48.33it/s, loss=tensor(1.00000e-02 *
 39%|███▉      | 123/312 [00:02<00:03, 47.95it/s, loss=tensor(1.00000e-02 *
 39%|███▉      | 123/312 [00:02<00:03, 47.52it/s, loss=tensor(1.00000e-02 *
 41%|████      | 128/312 [00:02<00:03, 49.25it/s, loss=tensor(1.00000e-02 *
 41%|████      | 128/312 [00:02<00:03, 48.91it/s, loss=tensor(1.00000e-02 *
 41%|████      | 128/312 [00:02<00:03, 48.60it/s, loss=tensor(1.00000e-02 *
 41%|████      | 128/312 [00:02<00:03, 48.29it/s, loss=tensor(1.00000e-02 *
 41%|████      | 128/312 [00:02<00:03, 47.96it/s, loss=tensor(1.00000e-02 *
 41%|████      | 128/312 [00:02<00:03, 47.70it/s, loss=tensor(1.00000e-02 *
 41%|████      | 128/312 [00:02<00:03, 47.30it/s, loss=tensor(1.00000e-02 *
 43%|████▎     | 134/312 [00:02<00:03, 49.29it/s, loss=tensor(1.00000e-02 *
 43%|████▎     | 134/312 [00:02<00:03, 49.00it/s, loss=tensor(1.00000e-02 *
 43%|████▎  

 70%|██████▉   | 217/312 [00:04<00:01, 50.18it/s, loss=tensor(1.00000e-02 *
 70%|██████▉   | 217/312 [00:04<00:01, 49.99it/s, loss=tensor(1.00000e-02 *
 70%|██████▉   | 217/312 [00:04<00:01, 49.73it/s, loss=tensor(1.00000e-02 *
 70%|██████▉   | 217/312 [00:04<00:01, 49.49it/s, loss=tensor(1.00000e-02 *
 70%|██████▉   | 217/312 [00:04<00:01, 49.32it/s, loss=tensor(1.00000e-02 *
 70%|██████▉   | 217/312 [00:04<00:01, 49.07it/s, loss=tensor(1.00000e-02 *
 70%|██████▉   | 217/312 [00:04<00:01, 48.87it/s, loss=tensor(1.00000e-02 *
 71%|███████▏  | 223/312 [00:04<00:01, 50.15it/s, loss=tensor(1.00000e-02 *
 71%|███████▏  | 223/312 [00:04<00:01, 49.98it/s, loss=tensor(1.00000e-02 *
 71%|███████▏  | 223/312 [00:04<00:01, 49.68it/s, loss=tensor(1.00000e-02 *
 71%|███████▏  | 223/312 [00:04<00:01, 49.55it/s, loss=tensor(1.00000e-02 *
 71%|███████▏  | 223/312 [00:04<00:01, 49.41it/s, loss=tensor(1.00000e-03 *
 71%|███████▏  | 223/312 [00:04<00:01, 49.25it/s, loss=tensor(1.00000e-03 *
 71%|███████

[9.      0.01627 0.01232]                                                                  



## Analyse Results
Here we analyse the results of our training. We'll do this by taking all of our existing training data and running it through the model, keeping a record of both it and the resulting prediction.

Once we have all the training data and the predictions, we'll need to convert back from the embedding offsets into category names. We can do that using our `categorical_lookups` object that we defined earlier. Each column corresponds to a given category set and each offset corresponds to a category within that set.

In [27]:
results = list(reversed(sorted([(data.trn_ds.cats[i], model(torch.tensor([data.trn_ds.cats[i]]), [])[0][0]) for i in range(len(data.trn_ds.cats))], key=lambda x: x[1])))

## How did we do?
Pretty well, it seems.

Notice above how most of the recent clicks went to things in "Politicians", "Beautiful Beaches" and "Artist & Thinkers". Elements from those categories all top the list here along with a few others.

In [28]:
for categoricals, result in results:
    print(*[categorical_lookups[categorical_columns[i]]['id'][x] for i, x in enumerate(categoricals)], result)

com.endlessm.celebrities.en Angela Merkel Politicians tensor(0.5073)
com.endlessm.history.en Lazarillo de Renaissance tensor(0.3820)
com.endlessm.celebrities.en Cristina Fernández Politicians tensor(0.3673)
com.endlessm.history.en Carolingian Empire Middle Ages tensor(0.3362)
com.endlessm.geography.en El Salvador Countries of the World tensor(0.3263)
com.endlessm.video_movement TOP 10 Track and Field tensor(0.3144)
com.endlessm.celebrities.en Academy Awards Events & Awards tensor(0.3100)
com.endlessm.celebrities.en Enrique Peña Politicians tensor(0.2879)
com.endlessm.celebrities.en Pablo Picasso Artists & Thinkers tensor(0.2827)
com.endlessm.celebrities.en Johnny Depp Actors tensor(0.2785)
com.endlessm.celebrities.en René Descartes Artists & Thinkers tensor(0.2643)
com.endlessm.animals.en Atlantic Canary Birds tensor(0.2637)
com.endlessm.video_movement Snowboarding Tricks Tricks tensor(0.2588)
com.endlessm.animals.en Domestic Canary Birds tensor(0.2457)
com.endlessm.celebrities.en Rafa

com.endlessm.travel.en Mount Kenya Africa tensor(0.1113)
com.endlessm.celebrities.en Nelson Mandela Politicians tensor(0.1112)
com.endlessm.celebrities.en Carlos Ruiz Athletes tensor(0.1107)
com.endlessm.animals.en Tiger Salamander Amphibians tensor(0.1096)
com.endlessm.celebrities.en Benjamin Franklin Artists & Thinkers tensor(0.1095)
com.endlessm.celebrities.en Maroon 5 Musicians tensor(0.1085)
com.endlessm.geography.en Saudi Arabia Countries of the World tensor(0.1082)
com.endlessm.history.en Industrial Revolution Industrial Revolution tensor(0.1078)
com.endlessm.celebrities.en Kobe Bryant Athletes tensor(0.1077)
com.endlessm.travel.en Petronas Towers Asia tensor(0.1077)
com.endlessm.travel.en Abraj Al Architecture tensor(0.1068)
com.endlessm.history.en Division of Industrial Revolution tensor(0.1065)
com.endlessm.celebrities.en Golden Globe Events & Awards tensor(0.1062)
com.endlessm.history.en Declaration of Europe tensor(0.1056)
com.endlessm.celebrities.en Woody Allen Artists & T

com.endlessm.travel.en Mount Erebus Polar tensor(1.00000e-02 *
       7.2589)
com.endlessm.history.en Maya Religion The Americas tensor(1.00000e-02 *
       7.2581)
com.endlessm.celebrities.en Leonardo DiCaprio Actors tensor(1.00000e-02 *
       7.2524)
com.endlessm.animals.en Ocean Sunfish Marine Animals tensor(1.00000e-02 *
       7.2490)
com.endlessm.history.en Roman Emperor Europe tensor(1.00000e-02 *
       7.2256)
com.endlessm.celebrities.en Marilyn Monroe Actors tensor(1.00000e-02 *
       7.1884)
com.endlessm.celebrities.en Jennifer Aniston Actors tensor(1.00000e-02 *
       7.1819)
com.endlessm.history.en Representative Democracy Modern and Contemporary Age tensor(1.00000e-02 *
       7.1530)
com.endlessm.animals.en Guppy  Marine Animals tensor(1.00000e-02 *
       7.1508)
com.endlessm.history.en Spanish American Modern and Contemporary Age tensor(1.00000e-02 *
       7.1413)
com.endlessm.history.en Napoleonic Wars Europe tensor(1.00000e-02 *
       7.1302)
com.endlessm.geogra

com.endlessm.animals.en Hoverfly  Insects tensor(1.00000e-02 *
       5.8386)
com.endlessm.geography.en Quetzaltenango  Latin America tensor(1.00000e-02 *
       5.8382)
com.endlessm.geography.en Argentina  Countries of the World tensor(1.00000e-02 *
       5.8335)
com.endlessm.travel.en Buenos Aires South America tensor(1.00000e-02 *
       5.8246)
com.endlessm.travel.en Port Lockroy Polar tensor(1.00000e-02 *
       5.8024)
com.endlessm.celebrities.en Mila Kunis Actors tensor(1.00000e-02 *
       5.8005)
com.endlessm.animals.en Hylidae  Amphibians tensor(1.00000e-02 *
       5.7991)
com.endlessm.animals.en Akita Dog Mammals tensor(1.00000e-02 *
       5.7800)
com.endlessm.history.en Inca Empire Old Age tensor(1.00000e-02 *
       5.7776)
com.endlessm.travel.en Las Leñas Snowscapes tensor(1.00000e-02 *
       5.7696)
com.endlessm.celebrities.en Matt Bomer Actors tensor(1.00000e-02 *
       5.7622)
com.endlessm.travel.en Chehel Sotoun Asia tensor(1.00000e-02 *
       5.7559)
com.endles

com.endlessm.celebrities.en Skrillex  Musicians tensor(1.00000e-02 *
       4.5484)
com.endlessm.travel.en Crater Lake North America tensor(1.00000e-02 *
       4.5432)
com.endlessm.travel.en Gardens of Europe tensor(1.00000e-02 *
       4.5372)
com.endlessm.animals.en Persian Cat Mammals tensor(1.00000e-02 *
       4.5277)
com.endlessm.travel.en Boudhanath  Asia tensor(1.00000e-02 *
       4.5232)
com.endlessm.travel.en Ball's Pyramid Oceania tensor(1.00000e-02 *
       4.5112)
com.endlessm.travel.en Caribbean Sea Central America tensor(1.00000e-02 *
       4.5092)
com.endlessm.travel.en Guanajuato City Historical Landmarks tensor(1.00000e-02 *
       4.5085)
com.endlessm.geography.en Oceania  Countries of the World tensor(1.00000e-02 *
       4.5000)
com.endlessm.travel.en Havelock Island Asia tensor(1.00000e-02 *
       4.4976)
com.endlessm.travel.en Abu Simbel Africa tensor(1.00000e-02 *
       4.4898)
com.endlessm.travel.en Saint Basil's Europe tensor(1.00000e-02 *
       4.4898)


com.endlessm.history.en Brazil  The Americas tensor(1.00000e-02 *
       3.7463)
com.endlessm.travel.en Ping An Architecture tensor(1.00000e-02 *
       3.7450)
com.endlessm.travel.en Mexican National North America tensor(1.00000e-02 *
       3.7392)
com.endlessm.travel.en Cabo San Beautiful Beaches tensor(1.00000e-02 *
       3.7385)
com.endlessm.geography.en Iran  Countries of the World tensor(1.00000e-02 *
       3.7359)
com.endlessm.travel.en Ipanema  Beautiful Beaches tensor(1.00000e-02 *
       3.7346)
com.endlessm.travel.en Fort San Central America tensor(1.00000e-02 *
       3.7300)
com.endlessm.animals.en Skunk  Mammals tensor(1.00000e-02 *
       3.7218)
com.endlessm.travel.en Malibu Beach Beautiful Beaches tensor(1.00000e-02 *
       3.7184)
com.endlessm.travel.en Shahdag Mountain Asia tensor(1.00000e-02 *
       3.7148)
com.endlessm.history.en Pope  Old Age tensor(1.00000e-02 *
       3.7000)
com.endlessm.travel.en Rousham House Europe tensor(1.00000e-02 *
       3.6898)
co

com.endlessm.travel.en Empire State North America tensor(1.00000e-02 *
       2.8783)
com.endlessm.travel.en Yu Garden Asia tensor(1.00000e-02 *
       2.8730)
com.endlessm.geography.en Estonia  Countries of the World tensor(1.00000e-02 *
       2.8727)
com.endlessm.travel.en Portugal  Countries tensor(1.00000e-02 *
       2.8674)
com.endlessm.travel.en Table Mountain Africa tensor(1.00000e-02 *
       2.8662)
com.endlessm.celebrities.en Evanescence  Musicians tensor(1.00000e-02 *
       2.8621)
com.endlessm.history.en Paleolithic  Prehistoric Age tensor(1.00000e-02 *
       2.8596)
com.endlessm.animals.en Olm  Amphibians tensor(1.00000e-02 *
       2.8560)
com.endlessm.animals.en Salamandridae  Amphibians tensor(1.00000e-02 *
       2.8503)
com.endlessm.animals.en Mandrill  Mammals tensor(1.00000e-02 *
       2.8482)
com.endlessm.animals.en Fly  Insects tensor(1.00000e-02 *
       2.8377)
com.endlessm.travel.en Kinkaku-ji  Asia tensor(1.00000e-02 *
       2.8278)
com.endlessm.travel.e

com.endlessm.travel.en Marrakesh  Africa tensor(1.00000e-02 *
       2.0249)
com.endlessm.geography.en Latvia  Countries of the World tensor(1.00000e-02 *
       2.0195)
com.endlessm.travel.en Laos  Asia tensor(1.00000e-02 *
       2.0177)
com.endlessm.geography.en Chile  Countries of the World tensor(1.00000e-02 *
       2.0176)
com.endlessm.geography.en Armenia  Countries of the World tensor(1.00000e-02 *
       2.0165)
com.endlessm.geography.en Greece  Countries of the World tensor(1.00000e-02 *
       2.0141)
com.endlessm.geography.en Romania  Countries of the World tensor(1.00000e-02 *
       2.0133)
com.endlessm.travel.en Chile  Countries tensor(1.00000e-02 *
       2.0090)
com.endlessm.travel.en Australia  Countries tensor(1.00000e-02 *
       1.9984)
com.endlessm.travel.en Stonehenge  Archeology & Ancient Culture tensor(1.00000e-02 *
       1.9823)
com.endlessm.history.en Jacobin  Renaissance tensor(1.00000e-02 *
       1.9688)
com.endlessm.travel.en Sanya  Asia tensor(1.00000e